In [1]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
from pycaret.classification import *

### Obtencion de los datos del clima para los periodos 2018 a 2022

Fuente: 

tabla con los nombres que tiene disponible la api: https://www.timeanddate.com/weather/usa

## Data de los vuelos

### Data Load

In [ ]:
data = pd.read_csv(
    'C:\\Users\\005184613\\Documents\\Documentacion Proyectos\\Python Proyects\\ML_Ops_udemy\\cookiecutter\\testiando_proyect_1\\data\\raw\\Flights_2018_1.csv', 
    low_memory=False
    )

data.head()

### Definicion del problema

**Variables de entrada**


- Fechas: Year, Month, DayofMonth, DayOfWeek, 

- Detalles de la aerolinea: Operating_Airline

- Detalles del origen: OriginAirportID (ID del aeropuerto), OriginStateName (Nombre del estado donde se encuentra ese aeropuerto)

- Detalles del destino: DestAirportID (ID del aeropuerto), DestStateName (Nombre del estado donde se encuentra ese aeropuerto)

- Detalles del vuelo SALIDA: CRSDepTime (tiempo estimado de salida)

- Detalles del vuelo Llegada: CRSArrTime (tiempo estimado de arrivo)

- Informacion sobre el vuelo: Flights (Cantidad de vuelos), Distance (Distancia entre aeropuertos |Millas|), DistanceGroup (Segmentos de distancia cada 250 millas)

**Posibles Variables de Salida** 

- Tiempos de salida desde origen: DepTime (tiempo actual de salida), DepDelay (Calculo de la demora salida), DepartureDelayGroups (agrupacion segun cuan demorada esta la salida)

- Tiempos de llegada a destino: ArrTime (tiempo actual de arrivo), ArrDelay (Calculo de la demora arrivo)

- Status vuelo: Cancelled (Si el vuelo esta cancellado), Diverted (Si el vuelo esta desviado), Duplicate (Flight swapped based on Form-3A data)

### Conocimiento de los datos

In [8]:
data.iloc[:5,15].name

'Operating_Airline '

> Dimensiones

In [9]:
print('El set de datos tiene {} registros y {} columnas'.format(data.shape[0],data.shape[1]))

El set de datos tiene 457688 registros y 120 columnas


> Columnas

In [10]:
data[~(data['Originally_Scheduled_Code_Share_Airline'].isnull())].iloc[:5, :10]

Year  Quarter  Month  DayofMonth  DayOfWeek  FlightDate  \
139413  2018        1      1          23          2  2018-01-23   
139414  2018        1      1          22          1  2018-01-22   
139415  2018        1      1          22          1  2018-01-22   
139416  2018        1      1          21          7  2018-01-21   
139417  2018        1      1          21          7  2018-01-21   

       Marketing_Airline_Network Operated_or_Branded_Code_Share_Partners  \
139413                        UA                            UA_CODESHARE   
139414                        UA                            UA_CODESHARE   
139415                        UA                            UA_CODESHARE   
139416                        UA                            UA_CODESHARE   
139417                        UA                            UA_CODESHARE   

        DOT_ID_Marketing_Airline IATA_Code_Marketing_Airline  
139413                     19977                          UA  
139414                     19977                          UA  
139415                     19977                          UA  
139416                     19977                          UA  
139417                     19977                          UA

In [11]:
data[~(data['Originally_Scheduled_Code_Share_Airline'].isnull())].iloc[:5, 10:20]

Flight_Number_Marketing_Airline  \
139413                             6265   
139414                             6131   
139415                             6088   
139416                             6278   
139417                             6265   

       Originally_Scheduled_Code_Share_Airline  \
139413                                      YV   
139414                                      YV   
139415                                      YV   
139416                                      YV   
139417                                      YV   

        DOT_ID_Originally_Scheduled_Code_Share_Airline  \
139413                                         20378.0   
139414                                         20378.0   
139415                                         20378.0   
139416                                         20378.0   
139417                                         20378.0   

       IATA_Code_Originally_Scheduled_Code_Share_Airline  \
139413                                                YV   
139414                                                YV   
139415                                                YV   
139416                                                YV   
139417                                                YV   

        Flight_Num_Originally_Scheduled_Code_Share_Airline Operating_Airline   \
139413                                             6265.0                  YX   
139414                                             6131.0                  YX   
139415                                             6088.0                  YX   
139416                                             6278.0                  YX   
139417                                             6265.0                  YX   

        DOT_ID_Operating_Airline IATA_Code_Operating_Airline Tail_Number  \
139413                     20452                          YX      N863RW   
139414                     20452                          YX      N633RW   
139415                     20452                          YX      N863RW   
139416                     20452                          YX      N863RW   
139417                     20452                          YX      N858RW   

        Flight_Number_Operating_Airline  
139413                             3289  
139414                             3286  
139415                             3288  
139416                             3295  
139417                             3290

In [12]:
data[~(data['Originally_Scheduled_Code_Share_Airline'].isnull())].iloc[:5, 20:30]

OriginAirportID  OriginAirportSeqID  OriginCityMarketID Origin  \
139413            13851             1385103               33851    OKC   
139414            15016             1501606               31123    STL   
139415            12266             1226603               31453    IAH   
139416            10397             1039707               30397    ATL   
139417            13851             1385103               33851    OKC   

           OriginCityName OriginState  OriginStateFips OriginStateName  \
139413  Oklahoma City, OK          OK               40        Oklahoma   
139414      St. Louis, MO          MO               29        Missouri   
139415        Houston, TX          TX               48           Texas   
139416        Atlanta, GA          GA               13         Georgia   
139417  Oklahoma City, OK          OK               40        Oklahoma   

        OriginWac  DestAirportID  
139413         73          12266  
139414         64          12266  
139415         74          13851  
139416         34          12266  
139417         73          12266

In [13]:
data[~(data['Originally_Scheduled_Code_Share_Airline'].isnull())].iloc[:5, 30:40]

DestAirportSeqID  DestCityMarketID Dest       DestCityName DestState  \
139413           1226603             31453  IAH        Houston, TX        TX   
139414           1226603             31453  IAH        Houston, TX        TX   
139415           1385103             33851  OKC  Oklahoma City, OK        OK   
139416           1226603             31453  IAH        Houston, TX        TX   
139417           1226603             31453  IAH        Houston, TX        TX   

        DestStateFips DestStateName  DestWac  CRSDepTime  DepTime  
139413             48         Texas       74         920    906.0  
139414             48         Texas       74         830    813.0  
139415             40      Oklahoma       73        1940   1951.0  
139416             48         Texas       74        1922   2117.0  
139417             48         Texas       74         920    910.0

In [14]:
data[~(data['Originally_Scheduled_Code_Share_Airline'].isnull())].iloc[:5, 40:50]

DepDelay  DepDelayMinutes  DepDel15  DepartureDelayGroups DepTimeBlk  \
139413     -14.0              0.0       0.0                  -1.0  0900-0959   
139414     -17.0              0.0       0.0                  -2.0  0800-0859   
139415      11.0             11.0       0.0                   0.0  1900-1959   
139416     115.0            115.0       1.0                   7.0  1900-1959   
139417     -10.0              0.0       0.0                  -1.0  0900-0959   

        TaxiOut  WheelsOff  WheelsOn  TaxiIn  CRSArrTime  
139413     16.0      922.0    1028.0     7.0        1052  
139414     16.0      829.0    1028.0     8.0        1051  
139415     13.0     2004.0    2110.0     8.0        2110  
139416     14.0     2131.0    2243.0     8.0        2045  
139417     13.0      923.0    1030.0     4.0        1052

In [15]:
data[~(data['Originally_Scheduled_Code_Share_Airline'].isnull())].iloc[:5, 50:60]

ArrTime  ArrDelay  ArrDelayMinutes  ArrDel15  ArrivalDelayGroups  \
139413   1035.0     -17.0              0.0       0.0                -2.0   
139414   1036.0     -15.0              0.0       0.0                -1.0   
139415   2118.0       8.0              8.0       0.0                 0.0   
139416   2251.0     126.0            126.0       1.0                 8.0   
139417   1034.0     -18.0              0.0       0.0                -2.0   

       ArrTimeBlk  Cancelled CancellationCode  Diverted  CRSElapsedTime  
139413  1000-1059        0.0              NaN       0.0            92.0  
139414  1000-1059        0.0              NaN       0.0           141.0  
139415  2100-2159        0.0              NaN       0.0            90.0  
139416  2000-2059        0.0              NaN       0.0           143.0  
139417  1000-1059        0.0              NaN       0.0            92.0

In [16]:
data[~(data['Originally_Scheduled_Code_Share_Airline'].isnull())].iloc[:5, 60:70]

ActualElapsedTime  AirTime  Flights  Distance  DistanceGroup  \
139413               89.0     66.0      1.0     395.0              2   
139414              143.0    119.0      1.0     667.0              3   
139415               87.0     66.0      1.0     395.0              2   
139416              154.0    132.0      1.0     689.0              3   
139417               84.0     67.0      1.0     395.0              2   

        CarrierDelay  WeatherDelay  NASDelay  SecurityDelay  LateAircraftDelay  
139413           NaN           NaN       NaN            NaN                NaN  
139414           NaN           NaN       NaN            NaN                NaN  
139415           NaN           NaN       NaN            NaN                NaN  
139416         115.0           0.0      11.0            0.0                0.0  
139417           NaN           NaN       NaN            NaN                NaN

> Estados, Destinos y Origenes

In [1]:
# Checkeo la cantidad de estados unicos en origen y destino. ¿Hay alguno que este en destino y no en origen?

#Guardo la cantidad de valores unicos en cada columna
# Variables: origStateCount, destStateCount
# Funcion unique(), parametros(No tiene), descrip: Esta funcion me devuelve una lista de los valores unicos de una pandas.Series.
origStateCount = data['OriginStateName'].unique().__len__()
destStateCount = data['DestStateName'].unique().__len__()

# Comparo si la cantidad es la misma
if origStateCount == destStateCount :
    print('La cantidad de estados de origen son {} y los de destino {}'.format(origStateCount, destStateCount))
    
    # Comparo si tenemos los mismos valores dentro de las columnas
    if sorted(data['OriginStateName'].unique()) == sorted(data['DestStateName'].unique()):
        print('\n')
        print('Los estados en ambas columnas son los mismos, podemos proceder con el encoding')
    # Aviso si no son iguales
    else:
        print('\n')
        print('Los estados en ambas columnas no son los mismos, OJO con el encoding')

# Aviso si no tienen la misma cantidad
else:
    print('OJO que las cantidades de estados no son iguales.')


NameError: name 'data' is not defined

In [18]:
# Chequeo de valores
data[data['DestStateName'].str.contains('Vir')]['DestStateName'].value_counts()

Virginia               14438
West Virginia            330
U.S. Virgin Islands      122
Name: DestStateName, dtype: int64

In [19]:
# Diccionario para los mapeos de los estados segun region lvl1
map_state_region_lvl1 = {
    'Connecticut':'Northeast', 
    'Maine':'Northeast', 
    'Massachusetts':'Northeast', 
    'New Hampshire':'Northeast', 
    'Rhode Island':'Northeast', 
    'Vermont':'Northeast',
    'New Jersey':'Northeast', 
    'New York':'Northeast', 
    'Pennsylvania':'Northeast',
    'Illinois':'Midwest', 
    'Indiana':'Midwest', 
    'Michigan':'Midwest', 
    'Ohio':'Midwest', 
    'Wisconsin':'Midwest',
    'Iowa':'Midwest', 
    'Kansas':'Midwest', 
    'Minnesota':'Midwest', 
    'Missouri':'Midwest', 
    'Nebraska':'Midwest', 
    'North Dakota':'Midwest', 
    'South Dakota':'Midwest',
    'Delaware':'South', 
    'Florida':'South', 
    'Georgia':'South', 
    'Maryland':'South', 
    'North Carolina':'South', 
    'South Carolina':'South', 
    'Virginia':'South', 
    'Washington':'South',
    'West Virginia':'South',
    'Alabama':'South', 
    'Kentucky':'South', 
    'Mississippi':'South', 
    'Tennessee':'South',
    'Arkansas':'South', 
    'Louisiana':'South', 
    'Oklahoma':'South', 
    'Texas':'South',
    'Arizona':'West', 
    'Colorado':'West', 
    'Idaho':'West', 
    'Montana':'West', 
    'Nevada':'West', 
    'New Mexico':'West', 
    'Utah':'West', 
    'Wyoming':'West',
    'Alaska':'West', 
    'California':'West', 
    'Hawaii':'West', 
    'Oregon':'West', 
    'Washington':'West',
    'Puerto Rico':'US Abroad',
    'U.S. Pacific Trust Territories and Possessions':'US Abroad',
    'U.S. Virgin Islands':'US Abroad'
}

# Diccionario para los mapeos de los estados segun region lvl2
map_state_region_lvl2 = {
    'Connecticut':'New England', 
    'Maine':'New England', 
    'Massachusetts':'New England', 
    'New Hampshire':'New England', 
    'Rhode Island':'New England', 
    'Vermont':'New England',
    'New Jersey':'Middle Atlantic', 
    'New York':'Middle Atlantic', 
    'Pennsylvania':'Middle Atlantic',
    'Illinois':'East North Central', 
    'Indiana':'East North Central', 
    'Michigan':'East North Central', 
    'Ohio':'East North Central', 
    'Wisconsin':'East North Central',
    'Iowa':'West North Central', 
    'Kansas':'West North Central', 
    'Minnesota':'West North Central', 
    'Missouri':'West North Central', 
    'Nebraska':'West North Central', 
    'North Dakota':'West North Central', 
    'South Dakota':'West North Central',
    'Delaware':'South Atlantic', 
    'Florida':'South Atlantic', 
    'Georgia':'South Atlantic', 
    'Maryland':'South Atlantic', 
    'North Carolina':'South Atlantic', 
    'South Carolina':'South Atlantic', 
    'Virginia':'South Atlantic', 
    'Washington':'South Atlantic',
    'West Virginia':'South Atlantic',
    'Alabama':'East South Central', 
    'Kentucky':'East South Central', 
    'Mississippi':'East South Central', 
    'Tennessee':'East South Central',
    'Arkansas':'West South Central', 
    'Louisiana':'West South Central', 
    'Oklahoma':'West South Central', 
    'Texas':'West South Central',
    'Arizona':'Mountain', 
    'Colorado':'Mountain', 
    'Idaho':'Mountain', 
    'Montana':'Mountain', 
    'Nevada':'Mountain', 
    'New Mexico':'Mountain', 
    'Utah':'Mountain', 
    'Wyoming':'Mountain',
    'Alaska':'Pacific', 
    'California':'Pacific', 
    'Hawaii':'Pacific', 
    'Oregon':'Pacific', 
    'Washington':'Pacific',
    'Puerto Rico':'US Abroad',
    'U.S. Pacific Trust Territories and Possessions':'US Abroad',
    'U.S. Virgin Islands':'US Abroad'
}

# Prueba de agregado de las variables state_reg_lvl1 y state_reg_lvl2
test_series = data['OriginStateName'].map(map_state_region_lvl2)

data[test_series.isnull()]['OriginStateName'].value_counts()

Series([], Name: OriginStateName, dtype: int64)

> Nulos

In [20]:
data.isnull().sum()

Year                                                       0
Quarter                                                    0
Month                                                      0
DayofMonth                                                 0
DayOfWeek                                                  0
FlightDate                                                 0
Marketing_Airline_Network                                  0
Operated_or_Branded_Code_Share_Partners                    0
DOT_ID_Marketing_Airline                                   0
IATA_Code_Marketing_Airline                                0
Flight_Number_Marketing_Airline                            0
Originally_Scheduled_Code_Share_Airline               457533
DOT_ID_Originally_Scheduled_Code_Share_Airline        457533
IATA_Code_Originally_Scheduled_Code_Share_Airline     457533
Flight_Num_Originally_Scheduled_Code_Share_Airline    457533
Operating_Airline                                          0
DOT_ID_Operating_Airline

> Numero de vuelo y Aerolinea

In [21]:
# Definiciones

# Si los numeros Flight_Number_Marketing_Airline son los mismos que Flight_Number_Operating_Airline el campo Flight_Num_Originally_Scheduled_Code_Share_Airline es nulo. 

# El numero de vuelo cambia cuando este se agenda a otra aerolinea. Es decir si viene con un Originally_Scheduled_Code_Share_Airline, quiere decir que antes el vuelo tenia otro Operating_Airline

data[data['Flight_Number_Marketing_Airline'] == data['Flight_Number_Operating_Airline']]['Flight_Num_Originally_Scheduled_Code_Share_Airline'].value_counts(dropna=False)

NaN    457533
Name: Flight_Num_Originally_Scheduled_Code_Share_Airline, dtype: int64

In [22]:
data[data['Flight_Number_Marketing_Airline'] == 6265][['Flight_Number_Marketing_Airline','Flight_Number_Operating_Airline','Flight_Num_Originally_Scheduled_Code_Share_Airline', 'Originally_Scheduled_Code_Share_Airline', 'Operating_Airline ', 'Flights', 'Diverted', 'Cancelled']]

Flight_Number_Marketing_Airline  Flight_Number_Operating_Airline  \
63447                              6265                             6265   
83685                              6265                             6265   
84202                              6265                             6265   
84466                              6265                             6265   
84736                              6265                             6265   
84967                              6265                             6265   
85213                              6265                             6265   
85472                              6265                             6265   
85727                              6265                             6265   
85988                              6265                             6265   
139413                             6265                             3289   
139417                             6265                             3290   
139425                             6265                             3286   
139441                             6265                             3209   
139447                             6265                             3286   
202499                             6265                             6265   
202767                             6265                             6265   
203015                             6265                             6265   
203243                             6265                             6265   
203491                             6265                             6265   
203746                             6265                             6265   
203998                             6265                             6265   
204260                             6265                             6265   
204515                             6265                             6265   
204774                             6265                             6265   
205022                             6265                             6265   
205259                             6265                             6265   
205474                             6265                             6265   
206308                             6265                             6265   
206551                             6265                             6265   
206814                             6265                             6265   
224614                             6265                             6265   
224881                             6265                             6265   
225882                             6265                             6265   
226178                             6265                             6265   
226474                             6265                             6265   
226771                             6265                             6265   
227069                             6265                             6265   
227372                             6265                             6265   
227769                             6265                             6265   
228041                             6265                             6265   
228042                             6265                             6265   
288806                             6265                             6265   

        Flight_Num_Originally_Scheduled_Code_Share_Airline  \
63447                                                 NaN    
83685                                                 NaN    
84202                                                 NaN    
84466                                                 NaN    
84736                                                 NaN    
84967                                                 NaN    
85213                                                 NaN    
85472                                                 NaN    
85727                                                 NaN    
85988                                      

## Procesamiento

###  Data Selection

In [23]:
# Lista con las columnas que tienen los datos que tendriamos a priori
columns_lists_inputs = [
    'Year', 
    'Month', 
    'DayofMonth', 
    'DayOfWeek', 
    'Originally_Scheduled_Code_Share_Airline', 
    'Operating_Airline ', 
    'OriginAirportID', 
    'OriginStateName', 
    'DestAirportID', 
    'DestStateName', 
    'CRSDepTime',  
    'CRSArrTime',  
    'Flights',
    'Distance',
    'DistanceGroup',
    'Cancelled'
]

# Lista con las posibles variables a estimar dependiendo del problema que abordemos
columns_lists_outputs = [
    'DepTime', 
    'DepDelay'
    'ArrTime', 
    'ArrDelay', 
    'Diverted', 
    'Duplicate'
]

# Nos quedamos con una copia de los datos que igresan, que contienen los atributos a priori
df_in = data[columns_lists_inputs].copy()

### Ingenieria de varibales

#### Regiones de los estados

Podemos agrepar los estados por la region en donde se encuentran y a su vez dividir esas regiones en algunas mas pequeñas.

> Fuente: [Wikipedia](https://en.wikipedia.org/wiki/List_of_regions_of_the_United_States)

In [24]:
# Diccionario para los mapeos de los estados segun region lvl1
map_state_region_lvl1 = {
    'Connecticut':'Northeast', 
    'Maine':'Northeast', 
    'Massachusetts':'Northeast', 
    'New Hampshire':'Northeast', 
    'Rhode Island':'Northeast', 
    'Vermont':'Northeast',
    'New Jersey':'Northeast', 
    'New York':'Northeast', 
    'Pennsylvania':'Northeast',
    'Illinois':'Midwest', 
    'Indiana':'Midwest', 
    'Michigan':'Midwest', 
    'Ohio':'Midwest', 
    'Wisconsin':'Midwest',
    'Iowa':'Midwest', 
    'Kansas':'Midwest', 
    'Minnesota':'Midwest', 
    'Missouri':'Midwest', 
    'Nebraska':'Midwest', 
    'North Dakota':'Midwest', 
    'South Dakota':'Midwest',
    'Delaware':'South', 
    'Florida':'South', 
    'Georgia':'South', 
    'Maryland':'South', 
    'North Carolina':'South', 
    'South Carolina':'South', 
    'Virginia':'South', 
    'Washington':'South',
    'West Virginia':'South',
    'Alabama':'South', 
    'Kentucky':'South', 
    'Mississippi':'South', 
    'Tennessee':'South',
    'Arkansas':'South', 
    'Louisiana':'South', 
    'Oklahoma':'South', 
    'Texas':'South',
    'Arizona':'West', 
    'Colorado':'West', 
    'Idaho':'West', 
    'Montana':'West', 
    'Nevada':'West', 
    'New Mexico':'West', 
    'Utah':'West', 
    'Wyoming':'West',
    'Alaska':'West', 
    'California':'West', 
    'Hawaii':'West', 
    'Oregon':'West', 
    'Washington':'West',
    'Puerto Rico':'US Abroad',
    'U.S. Pacific Trust Territories and Possessions':'US Abroad',
    'U.S. Virgin Islands':'US Abroad'
}

# Diccionario para los mapeos de los estados segun region lvl2
map_state_region_lvl2 = {
    'Connecticut':'New England', 
    'Maine':'New England', 
    'Massachusetts':'New England', 
    'New Hampshire':'New England', 
    'Rhode Island':'New England', 
    'Vermont':'New England',
    'New Jersey':'Middle Atlantic', 
    'New York':'Middle Atlantic', 
    'Pennsylvania':'Middle Atlantic',
    'Illinois':'East North Central', 
    'Indiana':'East North Central', 
    'Michigan':'East North Central', 
    'Ohio':'East North Central', 
    'Wisconsin':'East North Central',
    'Iowa':'West North Central', 
    'Kansas':'West North Central', 
    'Minnesota':'West North Central', 
    'Missouri':'West North Central', 
    'Nebraska':'West North Central', 
    'North Dakota':'West North Central', 
    'South Dakota':'West North Central',
    'Delaware':'South Atlantic', 
    'Florida':'South Atlantic', 
    'Georgia':'South Atlantic', 
    'Maryland':'South Atlantic', 
    'North Carolina':'South Atlantic', 
    'South Carolina':'South Atlantic', 
    'Virginia':'South Atlantic', 
    'Washington':'South Atlantic',
    'West Virginia':'South Atlantic',
    'Alabama':'East South Central', 
    'Kentucky':'East South Central', 
    'Mississippi':'East South Central', 
    'Tennessee':'East South Central',
    'Arkansas':'West South Central', 
    'Louisiana':'West South Central', 
    'Oklahoma':'West South Central', 
    'Texas':'West South Central',
    'Arizona':'Mountain', 
    'Colorado':'Mountain', 
    'Idaho':'Mountain', 
    'Montana':'Mountain', 
    'Nevada':'Mountain', 
    'New Mexico':'Mountain', 
    'Utah':'Mountain', 
    'Wyoming':'Mountain',
    'Alaska':'Pacific', 
    'California':'Pacific', 
    'Hawaii':'Pacific', 
    'Oregon':'Pacific', 
    'Washington':'Pacific',
    'Puerto Rico':'US Abroad',
    'U.S. Pacific Trust Territories and Possessions':'US Abroad',
    'U.S. Virgin Islands':'US Abroad'
}


# Creacion de las columnas con los mapeos
df_in['OriginStateRegion1'] = df_in['OriginStateName'].map(map_state_region_lvl1)
df_in['OriginStateRegion2'] = df_in['OriginStateName'].map(map_state_region_lvl2)
df_in['DestStateRegion1'] = df_in['DestStateName'].map(map_state_region_lvl1)
df_in['DestStateRegion2'] = df_in['DestStateName'].map(map_state_region_lvl2)

#### Tipos de viajes

La idea es crear una variable para identificar si el vuelo fue dentro del mismo estado, region del estado lvl1 o lvl2. Compararemos los campos de destino y origen que almacenan la informacion antes mencionada para crearlos.

In [25]:
df_in['sameStateFlight'] = (df_in['OriginStateName'] == df_in['DestStateName']).map({True:1, False:0})
df_in['sameStateRegion1Flight'] = (df_in['OriginStateRegion1'] == df_in['DestStateRegion1']).map({True:1, False:0})
df_in['sameStateRegion2Flight'] = (df_in['OriginStateRegion2'] == df_in['DestStateRegion2']).map({True:1, False:0})


In [26]:
df_in.head()

Year  Month  DayofMonth  DayOfWeek Originally_Scheduled_Code_Share_Airline  \
0  2018      1          23          2                                     NaN   
1  2018      1          24          3                                     NaN   
2  2018      1          25          4                                     NaN   
3  2018      1          26          5                                     NaN   
4  2018      1          27          6                                     NaN   

  Operating_Airline   OriginAirportID OriginStateName  DestAirportID  \
0                 9E            10146         Georgia          10397   
1                 9E            10146         Georgia          10397   
2                 9E            10146         Georgia          10397   
3                 9E            10146         Georgia          10397   
4                 9E            10146         Georgia          10397   

  DestStateName  CRSDepTime  CRSArrTime  Flights  Distance  DistanceGroup  \
0       Georgia        1202        1304      1.0     145.0              1   
1       Georgia        1202        1304      1.0     145.0              1   
2       Georgia        1202        1304      1.0     145.0              1   
3       Georgia        1202        1304      1.0     145.0              1   
4       Georgia        1400        1500      1.0     145.0              1   

   Cancelled OriginStateRegion1 OriginStateRegion2 DestStateRegion1  \
0        0.0              South     South Atlantic            South   
1        0.0              South     South Atlantic            South   
2        0.0              South     South Atlantic            South   
3        0.0              South     South Atlantic            South   
4        0.0              South     South Atlantic            South   

  DestStateRegion2  sameStateFlight  sameStateRegion1Flight  \
0   South Atlantic                1                       1   
1   South Atlantic                1                       1   
2   South Atlantic                1                       1   
3   South Atlantic                1                       1   
4   South Atlantic                1                       1   

   sameStateRegion2Flight  
0                       1  
1                       1  
2                       1  
3                       1  
4                       1

#### Cambios en la aerolinea

Se da la situacion que el viaje es realizado por una aerolinea normalmente, luego por alguna razon cambian la aeronea y el numero de vuelo cambia. Esto se puede identificar con el campo ´´Originally_Scheduled_Code_Share_Airline´´

In [27]:
df_in['AirlineChange'] = (df_in['Originally_Scheduled_Code_Share_Airline'].isnull()).map({False:1, True:0})

#### Cambios en los tipos de datos

**Variables a realizar label encoding**
- OriginStateName
- DestStateName
- OriginStateRegion1
- OriginStateRegion2
- DestStateRegion1
- DestStateRegion2


**Variables a pasar a Int**
- Flights
- Cancelled

In [28]:
df_in.dtypes

Year                                         int64
Month                                        int64
DayofMonth                                   int64
DayOfWeek                                    int64
Originally_Scheduled_Code_Share_Airline     object
Operating_Airline                           object
OriginAirportID                              int64
OriginStateName                             object
DestAirportID                                int64
DestStateName                               object
CRSDepTime                                   int64
CRSArrTime                                   int64
Flights                                    float64
Distance                                   float64
DistanceGroup                                int64
Cancelled                                  float64
OriginStateRegion1                          object
OriginStateRegion2                          object
DestStateRegion1                            object
DestStateRegion2               

In [29]:
"""
Variables a realizar label encoding
OriginStateName
DestStateName
OriginStateRegion1
OriginStateRegion2
DestStateRegion1
DestStateRegion2


Variables a pasar a Int
Flights
Cancelled
"""
# Pasar a Int
df_in[['Flights','Cancelled']] = df_in[['Flights','Cancelled']].astype(int)

In [30]:
# Trabajo pendiente (refactorizacion del codigo)

# 1. Hacer funciones hijas que sean llamadas por la funcion padre de label_encoding
# 2. Modificar la funcion padre para que puedas pasarle los nombres de las columnas y que sea util para todas las columnas


def label_encoding_state(dataframe, map_dict_state = 0):
    """
    --------------------------------------------------------------------
    FUNCION PARA EL LABEL Encoding
    --------------------------------------------------------------------
    Se encarga de mapear todas las etiquetas y las que no se encuentran
    dentro del mapeo iran a una clase generica que sera una especie de 
    no conocido.
    --------------------------------------------------------------------

    input: 
    - Pandas.DataFrame que contenga las columnas 'OriginStateName' y 'DestStateName'.
    - Para la primera vez, el diccionario no existe por lo tanto lo tenemos que crear
    y luego para ir aumentando lo obtendremos de un historico.

    output: 
    - Pandas.DataFrame que contiene las columnas mapeadas a numeros bajo los nombres 
    de 'OriginStateName_Label' y 'DestStateName_Label'
    - dict que contiene el mapeo

    """
    # 1. Checkeo la cantidad de estados unicos en origen y destino. ¿Hay alguno que este en destino y no en origen?

    #Guardo la cantidad de valores unicos en cada columna
    origStateCount = dataframe['OriginStateName'].unique().__len__()
    destStateCount = dataframe['DestStateName'].unique().__len__()

    # 2. Comparo si la cantidad es la misma
    if origStateCount == destStateCount:
        print('La cantidad de estados de origen son {} y los de destino {}'.format(origStateCount, destStateCount))
        
        # 3a Comparo si tenemos los mismos valores dentro de las columnas
        if sorted(dataframe['OriginStateName'].unique()) == sorted(dataframe['DestStateName'].unique()):
            print('\n')
            print('Los estados en ambas columnas son los mismos, procedemos con el encoding')

            # 4a Procedo con el label encoding
            # Mapeo los valores a un dicionario, en el caso de que no venga
            if map_dict_state == 0:
                map_dic = {clave:valor for valor, clave in enumerate(data['OriginStateName'].unique())}

            else:
                map_dic = map_dict_state
            
            # Uso .map() que mapea los valores conocido y los que no se encuentran en el dic los marca como NaN
            dataframe['OriginStateName_Label'] = dataframe['OriginStateName'].map(map_dic).replace(np.nan, 9999)
            dataframe['DestStateName_Label'] = dataframe['DestStateName'].map(map_dic).replace(np.nan, 9999)

            # 5 Envio los output
            print('\n')
            print('Se termino el Encoding!')
            return(dataframe, map_dic) 
        
        
        # 4b Aviso si no son iguales
        else:
            print('\n')
            print('Los estados en ambas columnas no son los mismos, OJO con el encoding')
            # 4a Procedo con el label encoding
            # Mapeo los valores a un dicionario, en el caso de que no venga
            if map_dict_state == 0:
                map_dic = {clave:valor for valor, clave in enumerate(data['OriginStateName'].unique())}

            else:
                map_dic = map_dict_state
            
            # Uso .map() que mapea los valores conocido y los que no se encuentran en el dic los marca como NaN
            dataframe['OriginStateName_Label'] = dataframe['OriginStateName'].map(map_dic).replace(np.nan, 9999)
            dataframe['DestStateName_Label'] = dataframe['DestStateName'].map(map_dic).replace(np.nan, 9999)

            # Creo la columna para ver los estados que no estan en Origen
            dataframe['IsInOriginStateName'] = dataframe['OriginStateName'].isin(dataframe['DestStateName'])

            # 5 Envio los output
            print('\n')
            print('Se termino el Encoding!')
            return(dataframe, map_dic)

    # 3b Aviso si no tienen la misma cantidad
    else:
        print('OJO que las cantidades de estados no son iguales.')
        # 4a Procedo con el label encoding
        # Mapeo los valores a un dicionario, en el caso de que no venga
        if map_dict_state == 0:
            map_dic = {clave:valor for valor, clave in enumerate(data['OriginStateName'].unique())}

        else:
            map_dic = map_dict_state
        
        # Uso .map() que mapea los valores conocido y los que no se encuentran en el dic los marca como NaN
        dataframe['OriginStateName_Label'] = dataframe['OriginStateName'].map(map_dic).replace(np.nan, 9999)
        dataframe['DestStateName_Label'] = dataframe['DestStateName'].map(map_dic).replace(np.nan, 9999)

        # Creo la columna para ver los estados que no estan en Origen
        dataframe['IsInOriginStateName'] = dataframe['OriginStateName'].isin(dataframe['DestStateName'])

        # 5 Envio los output
        print('\n')
        print('Se termino el Encoding!')
        return(dataframe, map_dic)
       


In [31]:
df_in, map_dict = label_encoding_state(df_in)

La cantidad de estados de origen son 52 y los de destino 52


Los estados en ambas columnas son los mismos, procedemos con el encoding


Se termino el Encoding!


In [32]:
# Creo los objetos de los label encoder 
le_region1 = preprocessing.LabelEncoder()
le_region2 = preprocessing.LabelEncoder()

# Entreno y tranformo las variables de Region1
df_in['OriginStateRegion1_label'] = le_region1.fit_transform(df_in['OriginStateRegion1'])
df_in['DestStateRegion1_label'] = le_region1.transform(df_in['DestStateRegion1'])

# Entreno y tranformo las variables de Region2
df_in['OriginStateRegion2_label'] = le_region2.fit_transform(df_in['OriginStateRegion2'])
df_in['DestStateRegion2_label'] = le_region2.transform(df_in['DestStateRegion2'])

### Pipeline Training

#### Balanceo de la variable de salida durante el split

- Variable de salida ´´Cancelled´´

In [33]:
df_in['Cancelled'].value_counts(normalize=True)

0    0.972274
1    0.027726
Name: Cancelled, dtype: float64

> OBSERVACION: Claramente la variable de salida esta desbalanceada, esto quiere decir que el modelo tendera a overfitiar la categoria con mas ejemplos y no generalizara bien.

Por este motivo para realizar el entrenamiento usaremos todos los ejemplos de vuelos cancelados y que estos representen el 40% de los ejemplos del dataset que luego separaremos en test y train

- Funcion de Split

In [34]:
# Separmos en data de train y test

def splitDataBasedCancelled(dataframe, percentaje):
    """
    --------------------------------------------------------------------
    FUNCION Split del dataframe en train y test balanceando Cancelled
    --------------------------------------------------------------------
    Se encarga de hacer el split con un porcentaje dado manteniendo la
    variables de salida cancelled balanceada. La idea es hacer 
    downsampling de los ejemplos de los vuelos no cancelado para no 
    sesgar al modelo.
    --------------------------------------------------------------------

    input: 
    - Pandas.DataFrame para separar.
    - Porcentaje de los datos usados para train.

    output: 
    - Pandas.DataFrame train
    - Pandas.DataFrame test

    """
    # 1. Obtengo el numero de ejemplos de la variable objetivo
    nCancellFlys = dataframe[dataframe['Cancelled']==1].shape[0]

    # 2. Separo Train
    dataTrainCan0 = dataframe[dataframe['Cancelled']==0].sample(n=int(nCancellFlys*percentaje))
    dataTrainCan1 = dataframe[dataframe['Cancelled']==1].sample(n=int(nCancellFlys*percentaje))

    # 3. Separo Test
    dataTestCan0 = dataframe[dataframe['Cancelled']==0][~(dataframe[dataframe['Cancelled']==0].index.isin(dataTrainCan0.index))].sample(n=int(nCancellFlys*(1-percentaje)))
    dataTestCan1 = dataframe[dataframe['Cancelled']==1][~(dataframe[dataframe['Cancelled']==1].index.isin(dataTrainCan1.index))]

    # 4. Armo los dataframe
    dfTrain = pd.concat([dataTrainCan0, dataTrainCan1])
    dfTest = pd.concat([dataTestCan0, dataTestCan1])

    # Exporto
    return(dfTrain, dfTest)


In [35]:
dataTrain, dataTest = splitDataBasedCancelled(df_in, 0.8)

> Validaciones rapidas

In [36]:
# Dimensiones Test
dataTrain.shape

(20304, 30)

In [37]:
# Dimensiones Train
dataTest.shape

(5075, 30)

In [38]:
# Que no compartan valores
dataTrain.isin(dataTest).sum()

Year                                       0
Month                                      0
DayofMonth                                 0
DayOfWeek                                  0
Originally_Scheduled_Code_Share_Airline    0
Operating_Airline                          0
OriginAirportID                            0
OriginStateName                            0
DestAirportID                              0
DestStateName                              0
CRSDepTime                                 0
CRSArrTime                                 0
Flights                                    0
Distance                                   0
DistanceGroup                              0
Cancelled                                  0
OriginStateRegion1                         0
OriginStateRegion2                         0
DestStateRegion1                           0
DestStateRegion2                           0
sameStateFlight                            0
sameStateRegion1Flight                     0
sameStateR

In [39]:
dataTest.isnull().sum()

Year                                          0
Month                                         0
DayofMonth                                    0
DayOfWeek                                     0
Originally_Scheduled_Code_Share_Airline    5073
Operating_Airline                             0
OriginAirportID                               0
OriginStateName                               0
DestAirportID                                 0
DestStateName                                 0
CRSDepTime                                    0
CRSArrTime                                    0
Flights                                       0
Distance                                      0
DistanceGroup                                 0
Cancelled                                     0
OriginStateRegion1                            0
OriginStateRegion2                            0
DestStateRegion1                              0
DestStateRegion2                              0
sameStateFlight                         

In [40]:
dataTrain['Cancelled'].value_counts(normalize=True)

0    0.5
1    0.5
Name: Cancelled, dtype: float64

In [41]:
dataTest['Cancelled'].value_counts(normalize=True)

1    0.500099
0    0.499901
Name: Cancelled, dtype: float64

In [42]:
dataTrain.dtypes

Year                                         int64
Month                                        int64
DayofMonth                                   int64
DayOfWeek                                    int64
Originally_Scheduled_Code_Share_Airline     object
Operating_Airline                           object
OriginAirportID                              int64
OriginStateName                             object
DestAirportID                                int64
DestStateName                               object
CRSDepTime                                   int64
CRSArrTime                                   int64
Flights                                      int32
Distance                                   float64
DistanceGroup                                int64
Cancelled                                    int32
OriginStateRegion1                          object
OriginStateRegion2                          object
DestStateRegion1                            object
DestStateRegion2               

### Model Trainnig

In [45]:
s = setup(dataTrain, target = 'Cancelled')

In [46]:
best = compare_models()